In [3]:
import wandb
wandb.login()


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\John Wilson\_netrc


True

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import pickle
import wandb

# Initialize a new wandb run
wandb.init(project='LLDA-Detecting Algal Blooms and Fish Kill')

# Load datasets with specified encoding
water_quality_df = pd.read_csv('Overall-dataset-llda(west phyto) assumed_knn_imputed_final2.csv', encoding='latin1')
weather_df = pd.read_csv('Overall-weather-dataset(csv).csv', encoding='iso-8859-1')

water_quality_df['Phytoplankton (cells/ml)'] = water_quality_df['Phytoplankton (cells/ml)'].str.replace(',', '')
water_quality_df['Month'] = water_quality_df['Month'].astype(str)

# Correct typos in the 'Month' column
water_quality_df['Month'] = water_quality_df['Month'].replace({
    'Febuary': 'February',
    'Aug': 'August',
    'Sept': 'September',
    'Nov': 'November',
    'Dec': 'December'
}, regex=False)

# List of columns to exclude from transformation
exclude_columns = ['Month', 'Wind', 'Condition']

# Function to remove non-numeric characters from each cell
def remove_non_numeric(value):
    if isinstance(value, str) and value not in exclude_columns:
        return re.sub(r'[^0-9.]', '', value)
    else:
        return value

# Apply the function to each cell in numeric columns of the dataframe
for column in weather_df.columns:
    if column not in exclude_columns:
        weather_df[column] = weather_df[column].apply(remove_non_numeric)

# Convert the numeric columns to numeric type
numeric_columns = [col for col in weather_df.columns if col not in exclude_columns]
weather_df[numeric_columns] = weather_df[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Define mappings for Wind column
wind_mapping = {
    'N': 1, 'NNE': 2, 'NE': 3, 'ENE': 4,
    'E': 5, 'ESE': 6, 'SE': 7, 'SSE': 8,
    'S': 9, 'SSW': 10, 'SW': 11, 'WSW': 12,
    'W': 13, 'WNW': 14, 'NW': 15, 'NNW': 16,
    'VAR': 17, 'CALM': 18
}

# Define mappings for Condition column
condition_mapping = {
    'Fair': 1, 'Mostly Cloudy': 2, 'Partly Cloudy': 3, 'Cloudy': 4,
    'Light Rain': 5, 'Light Rain Shower': 6, 'Rain': 7, 'Heavy Rain': 8,
    'Thunder': 9, 'Light Rain with Thunder': 10, 'T-Storm': 11,
    'Heavy Rain Shower': 12, 'Rain Shower': 13, 'Showers in the Vicinity': 14,
    'Thunder in the Vicinity': 15, 'Mostly Cloudy / Windy': 16,
    'Fair / Windy': 17, 'Partly Cloudy / Windy': 18, 'Rain / Windy': 19,
    'Light Rain Shower / Windy': 20, 'Heavy Rain / Windy': 21
}

# Apply mappings to Wind and Condition columns
weather_df['Wind'] = weather_df['Wind'].map(wind_mapping)
weather_df['Condition'] = weather_df['Condition'].map(condition_mapping)

# Define a function to compute the mode
def compute_mode(series):
    return series.mode().iloc[0] if not series.mode().empty else None

# Group by 'Year' and 'Month'
grouped = weather_df.groupby(['Year', 'Month'])

# Aggregate to get the mean for all columns except 'Wind' and 'Condition'
# and the mode for 'Wind' and 'Condition'
weather_monthly_stats = grouped.agg({
    'Wind': compute_mode,
    'Condition': compute_mode,
    'Time': 'mean',
    'Temperature': 'mean',
    'Dew Point' : 'mean',
    'Humidity': 'mean',
    'Wind Speed': 'mean',
    'Wind Gust': 'mean',
    'Pressure': 'mean',
    'Precip.': 'mean'
}).reset_index()

# Select relevant features and target
features = ['Temperature', 'Humidity', 'Wind', 'Wind Speed', 'Condition', 'pH (units)', 'Ammonia (mg/L)', 'Nitrate (mg/L)', 'Inorganic Phosphate (mg/L)', 'BOD (mg/l)', 'Dissolved Oxygen (mg/l)', 'Total coliforms (MPN/100ml)']
target = 'Phytoplankton (cells/ml)'

# Merge datasets on 'Month' and 'Year'
merged_df = pd.merge(water_quality_df, weather_monthly_stats, on=['Month', 'Year'])

# Split data into training and testing sets
X = merged_df[features]
y = merged_df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define parameter grid for Grid Search with SVR
param_grid = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [1, 10, 100],
    'epsilon': [0.1, 0.2, 0.3],
    'gamma': ['scale', 'auto']
}

# Initialize GridSearchCV with SVR
grid_search = GridSearchCV(SVR(), param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit GridSearchCV
grid_search.fit(X_train_scaled, y_train)

# Best parameters and score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score (Negative MSE):", best_score)

# Train model with best parameters
best_svr_model = grid_search.best_estimator_

# Make predictions and evaluate
y_pred_best_svr = best_svr_model.predict(X_test_scaled)
mse_best_svr = mean_squared_error(y_test, y_pred_best_svr)
mae_best_svr = mean_absolute_error(y_test, y_pred_best_svr)
r2_best_svr = r2_score(y_test, y_pred_best_svr)

print(f'Optimized SVR - Mean Squared Error: {mse_best_svr}')
print(f'Optimized SVR - Mean Absolute Error: {mae_best_svr}')
print(f'Optimized SVR - R^2 Score: {r2_best_svr}')

# Log hyperparameters and metrics to wandb
wandb.config.update(best_params)
wandb.log({
    'Best Score (Negative MSE)': best_score,
    'MSE': mse_best_svr,
    'MAE': mae_best_svr,
    'R2 Score': r2_best_svr
})

# Save the trained model to a file using pickle
model_filename = 'best_svr_model.pkl'
with open(model_filename, 'wb') as file:
    pickle.dump(best_svr_model, file)

print(f"Model saved to '{model_filename}'")

# Log the model artifact to wandb
artifact = wandb.Artifact('best_svr_model', type='model')
artifact.add_file(model_filename)
wandb.log_artifact(artifact)

# Finish the wandb run
wandb.finish()

# Example: Loading and using the saved model
with open(model_filename, 'rb') as file:
    loaded_svr_model = pickle.load(file)

# Predict using the loaded model
y_pred_loaded_model = loaded_svr_model.predict(X_test_scaled)


Best Parameters: {'C': 100, 'epsilon': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
Best Score (Negative MSE): -11167781236.216822
Optimized SVR - Mean Squared Error: 19309880777.458584
Optimized SVR - Mean Absolute Error: 52314.473318503435
Optimized SVR - R^2 Score: -0.07673343084866646
Model saved to 'best_svr_model.pkl'


Best Score (Negative MSE),▁
MAE,▁
MSE,▁
R2 Score,▁
Best Score (Negative MSE),-11167781236.21682
MAE,52314.47332
MSE,19309880777.45858
R2 Score,-0.07673
